In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st
import pingouin as pg

In [28]:
tratamentos = pd.read_csv('math_statistics_py/correlation/dados_tratamentos.csv', sep=';')
nps = pd.read_csv('math_statistics_py/correlation/nps_example(1).csv', sep=';')
bolsa = pd.read_csv('math_statistics_py/correlation/dados_bolsa.csv', sep=';', decimal=',')

In [29]:
tratamentos

,id,tratamento,evolucao
0,1,tratado,melhorou
1,2,tratado,melhorou
2,3,nao-tratado,melhorou
3,4,tratado,melhorou
4,5,tratado,nao-melhorou
...,...,...,...
100,101,tratado,melhorou
101,102,tratado,melhorou
102,103,nao-tratado,nao-melhorou
103,104,tratado,melhorou


In [30]:
bolsa

,data,petr4,bbdc3,vale5,ambv4,itub4
0,2004-05-26,-0.002270,0.009524,0.013699,0.032668,0.009843
1,2004-05-27,0.026301,0.028396,0.028303,0.014060,0.021442
2,2004-05-28,-0.016484,-0.000092,-0.025261,0.019151,0.000000
3,2004-05-31,0.010986,-0.009174,0.011235,-0.001786,0.000000
4,2004-06-01,0.028142,0.023056,-0.004518,0.013629,0.012107
...,...,...,...,...,...,...
1773,2011-03-14,0.003905,-0.000407,-0.002139,-0.010953,0.004765
1774,2011-03-15,-0.008487,0.004473,-0.016724,-0.004873,0.011994
1775,2011-03-16,-0.009272,-0.017409,-0.025294,-0.020699,-0.023705
1776,2011-03-17,0.010799,-0.001236,0.027293,-0.001136,-0.005364


In [31]:
nps

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male
...,...,...,...,...,...,...
2574,13640772,Complete,3 years to less than 5 years,18-24,10.0,Female
2575,13652344,Incomplete,1 year to less than 3 years,35-44,NaN,NaN
2576,13732056,Complete,1 year to less than 3 years,18-24,10.0,Female
2577,13734055,Complete,Less than 6 months,25-34,10.0,Male


##  Coeficientes de associação

### Coeficiente de phi

In [32]:
crosstb = pd.crosstab(tratamentos['tratamento'], tratamentos['evolucao'])

crosstb

evolucao,melhorou,nao-melhorou
tratamento,,
nao-tratado,26,29
tratado,35,15


In [33]:
# obtain qui quadrado e phi
qui_quadrado = st.chi2_contingency(crosstb)[0]

n= np.sum(np.sum(crosstb))

Phi = np.sqrt(qui_quadrado/n)
Phi

/home/borges/miniconda3/envs/py312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


np.float64(0.21072588592155886)

In [34]:
qui, p, gl, t_esp = st.chi2_contingency(crosstb)

# qui quadrado e p-value
qui, p

(np.float64(4.6625668947297125), np.float64(0.030827072412198585))

### Coeficiente v de cramer

In [40]:
# limpeza dos dados
nps_filtrado = nps[(nps['response_status'] == 'Complete') & \
                                (nps['nps_score'].notna()) & \
                                (nps['gender'].notna())].reset_index()

In [41]:
def create_nps_groups(x):
    if x <= 5:
        return 'detratores'
    elif x >= 9 :
        return 'promotores'
    else:
        return 'neutros'

nps_filtrado['nps_groups'] = nps_filtrado['nps_score'].apply(lambda x: create_nps_groups(x))

nps_filtrado.groupby('nps_groups').size().to_frame('n')

,n
nps_groups,
detratores,25
neutros,171
promotores,2047


In [42]:
crosstb = pd.crosstab(nps_filtrado['gender'], nps_filtrado['nps_groups'])

crosstb

nps_groups,detratores,neutros,promotores
gender,,,
Female,12,105,1565
Male,13,66,482


In [48]:
vcramer1 = st.contingency.association(crosstb, method='cramer')
vcramer1

0.1134646600588705

In [53]:
qui_quadrado = st.chi2_contingency(crosstb)[0]

m = np.sum(np.sum(crosstb))

minshape = min(crosstb.shape)-1

vcramer2 = np.sqrt(qui_quadrado / (n * minshape))
vcramer2

np.float64(0.1134646600588705)

In [54]:
# teste qui quadrado
qui, p, gl, t_esp = st.chi2_contingency(crosstb)
qui, p

(np.float64(28.87689583154292), np.float64(5.363666207418583e-07))

## Coeficientes de correlação

In [55]:
bolsa[bolsa.isnull()]

,data,petr4,bbdc3,vale5,ambv4,itub4
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1773,NaN,NaN,NaN,NaN,NaN,NaN
1774,NaN,NaN,NaN,NaN,NaN,NaN
1775,NaN,NaN,NaN,NaN,NaN,NaN
1776,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
bolsa.dropna()

,data,petr4,bbdc3,vale5,ambv4,itub4
0,2004-05-26,-0.002270,0.009524,0.013699,0.032668,0.009843
1,2004-05-27,0.026301,0.028396,0.028303,0.014060,0.021442
2,2004-05-28,-0.016484,-0.000092,-0.025261,0.019151,0.000000
3,2004-05-31,0.010986,-0.009174,0.011235,-0.001786,0.000000
4,2004-06-01,0.028142,0.023056,-0.004518,0.013629,0.012107
...,...,...,...,...,...,...
1773,2011-03-14,0.003905,-0.000407,-0.002139,-0.010953,0.004765
1774,2011-03-15,-0.008487,0.004473,-0.016724,-0.004873,0.011994
1775,2011-03-16,-0.009272,-0.017409,-0.025294,-0.020699,-0.023705
1776,2011-03-17,0.010799,-0.001236,0.027293,-0.001136,-0.005364


### Correlação pearson

In [60]:
pearson_corr = bolsa.corr(numeric_only=True)

pearson_corr

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.539247,0.724023,0.392074,0.593834
bbdc3,0.539247,1.000000,0.592143,0.470529,0.778506
vale5,0.724023,0.592143,1.000000,0.482919,0.642838
ambv4,0.392074,0.470529,0.482919,1.000000,0.488886
itub4,0.593834,0.778506,0.642838,0.488886,1.000000


In [63]:
pearson_corr.style.background_gradient(cmap='coolwarm')

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.539247,0.724023,0.392074,0.593834
bbdc3,0.539247,1.000000,0.592143,0.470529,0.778506
vale5,0.724023,0.592143,1.000000,0.482919,0.642838
ambv4,0.392074,0.470529,0.482919,1.000000,0.488886
itub4,0.593834,0.778506,0.642838,0.488886,1.000000


In [64]:
corr_cols = ['petr4','bbdc3','vale5','ambv4','itub4']

pg.pairwise_corr(bolsa, columns=corr_cols, method='pearson')

,X,Y,method,alternative,n,r,CI95%,p-unc,BF10,power
0,petr4,bbdc3,pearson,two-sided,1604,0.539247,"[0.5, 0.57]",1.094616e-121,7.488e+117,1.0
1,petr4,vale5,pearson,two-sided,1604,0.724023,"[0.7, 0.75]",1.122582e-260,3.654e+256,1.0
2,petr4,ambv4,pearson,two-sided,1604,0.392074,"[0.35, 0.43]",4.390573e-60,3.057e+56,1.0
3,petr4,itub4,pearson,two-sided,1604,0.593834,"[0.56, 0.62]",1.793002e-153,3.791e+149,1.0
4,bbdc3,vale5,pearson,two-sided,1604,0.592143,"[0.56, 0.62]",2.143940e-152,3.189e+148,1.0
5,bbdc3,ambv4,pearson,two-sided,1604,0.470529,"[0.43, 0.51]",3.714265e-89,2.774e+85,1.0
6,bbdc3,itub4,pearson,two-sided,1604,0.778506,"[0.76, 0.8]",0.000000e+00,inf,1.0
7,vale5,ambv4,pearson,two-sided,1604,0.482919,"[0.44, 0.52]",1.738600e-94,5.688e+90,1.0
8,vale5,itub4,pearson,two-sided,1604,0.642838,"[0.61, 0.67]",1.064830e-187,5.347e+183,1.0
9,ambv4,itub4,pearson,two-sided,1604,0.488886,"[0.45, 0.53]",3.929767e-97,2.467e+93,1.0


### Correlação Spearman 

In [71]:
# correlacao
spearman_correl = bolsa.corr(numeric_only=True ,method='spearman')
spearman_correl

# matriz ajustada
spearman_correl.style.background_gradient(cmap='coolwarm')

# colunas para correlacionar
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']

# teste
pg.pairwise_corr(bolsa, 
                columns = correl_columns, 
                method = 'spearman')

,X,Y,method,alternative,n,r,CI95%,p-unc,power
0,petr4,bbdc3,spearman,two-sided,1604,0.472143,"[0.43, 0.51]",7.723804e-90,1.0
1,petr4,vale5,spearman,two-sided,1604,0.644211,"[0.61, 0.67]",9.494683e-189,1.0
2,petr4,ambv4,spearman,two-sided,1604,0.330002,"[0.29, 0.37]",4.669599e-42,1.0
3,petr4,itub4,spearman,two-sided,1604,0.531198,"[0.5, 0.57]",1.762927e-117,1.0
4,bbdc3,vale5,spearman,two-sided,1604,0.505002,"[0.47, 0.54]",1.513500e-104,1.0
5,bbdc3,ambv4,spearman,two-sided,1604,0.419052,"[0.38, 0.46]",3.166567e-69,1.0
6,bbdc3,itub4,spearman,two-sided,1604,0.758946,"[0.74, 0.78]",8.700692e-301,1.0
7,vale5,ambv4,spearman,two-sided,1604,0.404373,"[0.36, 0.44]",3.794800e-64,1.0
8,vale5,itub4,spearman,two-sided,1604,0.574199,"[0.54, 0.61]",2.395419e-141,1.0
9,ambv4,itub4,spearman,two-sided,1604,0.449118,"[0.41, 0.49]",1.896860e-80,1.0


### Correlação Kendall

In [74]:
# correlacao
kendall_correl = bolsa.corr(numeric_only=True, method='kendall')
kendall_correl

# matriz ajustada
kendall_correl.style.background_gradient(cmap='coolwarm')

# colunas para correlacionar
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']

# teste
pg.pairwise_corr(bolsa, 
                columns = correl_columns, 
                method = 'kendall')

,X,Y,method,alternative,n,r,CI95%,p-unc,power
0,petr4,bbdc3,kendall,two-sided,1604,0.332691,"[0.29, 0.38]",1.358166e-88,1.0
1,petr4,vale5,kendall,two-sided,1604,0.471033,"[0.43, 0.51]",9.805061e-176,1.0
2,petr4,ambv4,kendall,two-sided,1604,0.231311,"[0.18, 0.28]",8.582460e-44,1.0
3,petr4,itub4,kendall,two-sided,1604,0.378755,"[0.34, 0.42]",2.531465e-114,1.0
4,bbdc3,vale5,kendall,two-sided,1604,0.358557,"[0.32, 0.4]",1.375615e-102,1.0
5,bbdc3,ambv4,kendall,two-sided,1604,0.294492,"[0.25, 0.34]",8.231269e-70,1.0
6,bbdc3,itub4,kendall,two-sided,1604,0.571953,"[0.54, 0.6]",7.030155e-258,1.0
7,vale5,ambv4,kendall,two-sided,1604,0.282942,"[0.24, 0.33]",1.246269e-64,1.0
8,vale5,itub4,kendall,two-sided,1604,0.411663,"[0.37, 0.45]",1.100820e-134,1.0
9,ambv4,itub4,kendall,two-sided,1604,0.318638,"[0.27, 0.36]",1.852468e-81,1.0
